INSTALLING ALL THE REQUIRED PACKAGES

In [ ]:
!pip install google-auth
!pip install google-auth-oauthlib
!pip install google-auth-httplib2
!pip install google-api-python-client
!pip install pathway
!pip install sentence-transformers
!pip install pandas
!pip install scikit-learn
!pip install PyPDF2
!pip install tensorflow
!pip install imbalanced-learn
!pip install pymupdf
!pip install higher

IMPORTING PACKAGES

In [ ]:
import requests
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
import re
from transformers import AutoTokenizer, AutoModel, pipeline
import torch
import os
import pandas as pd
import PyPDF2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import fitz
import spacy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from google.colab import drive

MOUNTING DRIVE

In [ ]:

drive.mount('/content/drive')

INTIALIZING THE PATHS

In [ ]:
# Define folder paths based on your Google Drive structure
base_path = '/content/drive/MyDrive/Colab Notebooks/Research_paper'  # base path
references_path = os.path.join(base_path, 'Reference')  # Path to references folder
papers_path = os.path.join(base_path, 'Papers')  # Path to papers folder

# Verify folder structure and files
papers_files = os.listdir(papers_path)  # List all papers in the papers folder
references_files = os.listdir(references_path)  # List all references in the references folder

# Display a few files from each folder to verify
papers_files[:5], references_files[:5]


-------------------------------------**TASK 1**-----------------------------------

MASTER FEATURE EXTRACTION AND FLATTENINF IT

In [ ]:
# 1. Text Extraction from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# 2. Section-wise Segmentation
def segment_text(text):
    # Basic segmentation based on common section headings in research papers
    sections = {
        "abstract": "",
        "introduction": "",
        "methodology": "",
        "results": "",
        "conclusion": ""
    }
    current_section = None
    for line in text.split("\n"):
        line = line.strip().lower()
        if "abstract" in line:
            current_section = "abstract"
        elif "introduction" in line:
            current_section = "introduction"
        elif "methodology" in line or "materials and methods" in line:
            current_section = "methodology"
        elif "results" in line:
            current_section = "results"
        elif "conclusion" in line:
            current_section = "conclusion"
        if current_section:
            sections[current_section] += line + " "
    return sections

# 3. TF-IDF Features
def extract_tfidf_features(texts, max_features=5000):
    vectorizer = TfidfVectorizer(stop_words="english", max_features=max_features)
    tfidf_matrix = vectorizer.fit_transform(texts)
    return tfidf_matrix, vectorizer.get_feature_names_out()

# 4. Semantic Features using BERT
def extract_bert_embeddings(text, model_name="bert-base-uncased"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**tokens)
    # Use the mean of the last hidden states as the embedding
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)
    return embeddings.numpy()

# 5. Sentence Coherence Features
def compute_sentence_coherence(text):
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    sentences = text.split(".")
    embeddings = model.encode(sentences)
    coherence_score = np.mean([
        np.dot(embeddings[i], embeddings[i+1]) /
        (np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[i+1]))
        for i in range(len(embeddings) - 1)
    ])
    return coherence_score

# 6. Numerical Feature Extraction (e.g., Metrics Validation)

def extract_numerical_features(text):
    """
    Extract numerical features from the given text.
    This function extracts all numbers from the text and returns a dictionary of numerical features.
    """
    # Extract all numbers from the text
    numbers = re.findall(r"\b\d+(\.\d+)?\b", text)

    # Filter out any empty strings or non-numeric values
    numbers = [float(num) for num in numbers if num.strip() != ""]

    # If no numbers are found, return default values
    if not numbers:
        return {"mean": None, "std": None, "max": None, "min": None, "count": 0}

    # Calculate and return basic numerical features
    return {
        "mean": sum(numbers) / len(numbers),
        "std": (sum((x - sum(numbers) / len(numbers))**2 for x in numbers) / len(numbers))**0.5,
        "max": max(numbers),
        "min": min(numbers),
        "count": len(numbers)  # Count of numbers extracted
    }

# 7. Logical Consistency Check
def check_logical_consistency(methodology, results):
    # Compare sentences from methodology and results for semantic alignment
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    methodology_embedding = model.encode([methodology])
    results_embedding = model.encode([results])
    similarity = np.dot(methodology_embedding, results_embedding.T) / (
        np.linalg.norm(methodology_embedding) * np.linalg.norm(results_embedding)
    )
    return similarity[0][0]

# Main Feature Extraction Pipeline
def extract_features(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    sections = segment_text(text)

    features = {}

    # TF-IDF Features
    features["tfidf"], feature_names = extract_tfidf_features([text])

    # BERT Embeddings
    features["bert_abstract"] = extract_bert_embeddings(sections.get("abstract", ""))
    features["bert_methodology"] = extract_bert_embeddings(sections.get("methodology", ""))
    features["bert_results"] = extract_bert_embeddings(sections.get("results", ""))

    # Sentence Coherence
    features["coherence_abstract"] = compute_sentence_coherence(sections.get("abstract", ""))
    features["coherence_methodology"] = compute_sentence_coherence(sections.get("methodology", ""))

    # Numerical Features
    features["numerical_abstract"] = extract_numerical_features(sections.get("abstract", ""))
    features["numerical_results"] = extract_numerical_features(sections.get("results", ""))

    # Logical Consistency
    features["logical_consistency"] = check_logical_consistency(
        sections.get("methodology", ""), sections.get("results", "")
    )

    return features




In [ ]:
def flatten_features(features_dict):
    """
    Flatten the dictionary into a list of features.
    """
    flattened = []

    # Add coherence features
    flattened.append(features_dict.get('coherence_abstract', None))
    flattened.append(features_dict.get('coherence_methodology', None))
    flattened.append(features_dict.get('logical_consistency', None))

    # Add numerical features for abstract and results, using .get() to avoid KeyError
    flattened.extend([
        features_dict.get('numerical_abstract', {}).get('mean', None),
        features_dict.get('numerical_abstract', {}).get('max', None),
        features_dict.get('numerical_abstract', {}).get('min', None),
        features_dict.get('numerical_abstract', {}).get('count', 0),  # Default to 0 if 'count' is missing

        features_dict.get('numerical_results', {}).get('mean', None),
        features_dict.get('numerical_results', {}).get('max', None),
        features_dict.get('numerical_results', {}).get('min', None),
        features_dict.get('numerical_results', {}).get('count', 0)  # Default to 0 if 'count' is missing
    ])

    # Add BERT embeddings, flattened to 1D array
    flattened.extend(features_dict.get('bert_abstract', {}).flatten() if 'bert_abstract' in features_dict else [])
    flattened.extend(features_dict.get('bert_methodology', {}).flatten() if 'bert_methodology' in features_dict else [])
    flattened.extend(features_dict.get('bert_results', {}).flatten() if 'bert_results' in features_dict else [])

    return flattened


EXTARCTING PAPERS TO BE PREDICTED

In [ ]:
# Extract text and paper IDs from all papers
papers_data = []
c = 0 #for counting file
for paper_file in papers_files:
    c += 1
    print(f"Processing paper {c}: {paper_file}")
    paper_path = os.path.join(papers_path, paper_file)

    if paper_file.endswith('.pdf'):  # Process only PDF files
        # Extract paper ID by removing '.pdf'
        paper_id = paper_file.replace('.pdf', '')

        # Extract features and flatten
        features_dict = extract_features(paper_path)  # Extract features from the PDF
        flattened_features = flatten_features(features_dict)
        flattened_features = np.array(flattened_features, dtype=np.float32)

        # Replace NaN values with 0
        if np.any(np.isnan(flattened_features)):
            print(f"Warning: NaN values found in {paper_path}. Replacing with 0.")
            for i in range(len(flattened_features)):  # Handle NaNs with a loop
                if np.isnan(flattened_features[i]):
                    flattened_features[i] = 0

        # Append data as a dictionary with paper ID and features
        papers_data.append({
            "paper_id": paper_id,
            "features": flattened_features
        })

# Extract `features` for model input
X_predict = np.array([paper["features"] for paper in papers_data])

# Display the paper ID and first 500 characters of features from the first paper
print(f"Paper ID: {papers_data[0]['paper_id']}")
print(f"First 500 features: {papers_data[0]['features'][:500]}")


EXTRACTING REFERENCE PAPERS ALONG WITH LABELS

In [ ]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder


# Function to get texts and labels from the 'References' folder
def extract_data_from_references(references_path):
    features_list = []
    labels = []

    # Traverse the 'publishable' folder and assign label 1 (Publishable)
    publishable_folder = os.path.join(references_path, 'Publishable')
    for category in os.listdir(publishable_folder):
        category_folder = os.path.join(publishable_folder, category)
        if os.path.isdir(category_folder):
            for paper in os.listdir(category_folder):
                if paper.endswith('.pdf'):
                    pdf_path = os.path.join(category_folder, paper)
                    features_dict = extract_features(pdf_path)
                    flattened_features = flatten_features(features_dict)

                    # Convert flattened_features to a NumPy array of floats
                    flattened_features = np.array(flattened_features, dtype=np.float32)

                    # Replace NaN values with 0 or a specified value
                    if np.any(np.isnan(flattened_features)):
                        print(f"Warning: NaN values found in {pdf_path}. Replacing with 0.")
                        flattened_features = np.nan_to_num(flattened_features, nan=0)

                    features_list.append(flattened_features)
                    labels.append(1)  # Label 1 for publishable papers

    # Traverse the 'non_publishable' folder and assign label 0 (Non-Publishable)
    non_publishable_folder = os.path.join(references_path, 'Non-Publishable')
    for paper in os.listdir(non_publishable_folder):
        if paper.endswith('.pdf'):
            pdf_path = os.path.join(non_publishable_folder, paper)
            features_dict = extract_features(pdf_path)
            flattened_features = flatten_features(features_dict)

            # Convert flattened_features to a NumPy array of floats
            flattened_features = np.array(flattened_features, dtype=np.float32)

            # Replace NaN values with 0 or a specified value
            if np.any(np.isnan(flattened_features)):
                print(f"Warning: NaN values found in {pdf_path}. Replacing with 0.")
                flattened_features = np.nan_to_num(flattened_features, nan=0)

            features_list.append(flattened_features)
            labels.append(0)  # Label 0 for non-publishable papers

    return features_list, labels


# Extract texts and labels from the references folder
features_list, labels = extract_data_from_references(references_path)

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Print the first few entries to check
print("Texts and Labels for Training:")
print(features_list[:2])  # First 2 features (for example)
print(labels[:2])  # Corresponding labels for the first 2 papers


DIVIDING THE EXTRACTED FEATURES LIST INTO TRAINING AND TESTING DATA

In [ ]:
X = np.array(features_list)
y = np.array(labels)
# Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


# Print shapes of the resulting sets to verify
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {len(y_train)}")
print(f"Shape of y_test: {len(y_test)}")

TRAINING A FEED-FORWARD NEURAL NETWORK USING MODEL-AGNOSTIC META-LEARNING (MAML) TO ADAPT QUICKLY TO NEW TASKS, WHERE THE MODEL IS UPDATED USING TASK-SPECIFIC DATA, FOLLOWED BY META-GRADIENT UPDATES TO OPTIMIZE THE MODEL'S GENERALIZATION ABILITY ON UNSEEN TASKS.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import higher

# Define a simple feed-forward neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(2315, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 2)  # Output layer for binary classification

        # Dropout layers to prevent overfitting
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.relu(self.fc3(x))
        x = self.dropout(x)
        x = torch.relu(self.fc4(x))
        x = self.dropout(x)
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x


In [ ]:
def maml_update(model, X_batch, y_batch, lr=0.01):
    model.zero_grad()

    # Forward pass through the model
    outputs = model(X_batch)
    loss = nn.CrossEntropyLoss()(outputs, y_batch)

    # Backpropagation and gradient computation
    loss.backward()

    for param in model.parameters():
        param.data -= lr * param.grad  # Parameter update step for MAML

    return loss

def meta_train(model, train_data, test_data, val_data, meta_lr=0.001, lr=0.01, num_steps=10):
    optimizer = optim.Adam(model.parameters(), lr=meta_lr)

    for step in range(num_steps):
        # Adapt to the task using MAML update
        meta_train_batch, meta_test_batch = train_data, test_data
        maml_update(model, meta_train_batch[0], meta_train_batch[1], lr=lr)

        # Test the adapted model on the test batch (meta-test)
        test_outputs = model(meta_test_batch[0])
        test_loss = nn.CrossEntropyLoss()(test_outputs, meta_test_batch[1])

        # Meta-gradient update
        optimizer.zero_grad()
        test_loss.backward()
        optimizer.step()

        # Calculate and display validation accuracy
        with torch.no_grad():
            # Assuming val_data is a tuple (X_val, y_val)
            val_outputs = model(val_data[0])  # Forward pass on validation set
            val_predictions = torch.argmax(val_outputs, dim=1)  # Get the predicted class
            val_accuracy = (val_predictions == val_data[1]).float().mean().item()  # Calculate accuracy

        # Print progress
        print(f"Step {step + 1}/{num_steps} - Validation Accuracy: {val_accuracy * 100:.2f}%")

    return model


TRAINING AND TESTING THE MODELS

In [ ]:
# Initialize model
model = NeuralNetwork()

# Meta-train with MAML (Make sure meta_train is correctly implemented)
train_data = (torch.tensor(X_train[:1]), torch.tensor(y_train[:1]))  # Small subset for task-based learning
test_data = (torch.tensor(X_test[:2]), torch.tensor(y_test[:2]))
val_data = (torch.tensor(X_test[2:]), torch.tensor(y_test[2:]))

# Perform meta-training with MAML (Ensure proper evaluation is inside meta_train)
modell = meta_train(model, train_data, test_data,val_data, meta_lr=0.001, lr=0.01, num_steps=30)

#  Replace NaN values
X_train[np.isnan(X_train)] = 0
X_test[np.isnan(X_test)] = 0

# Alternative: Replace NaNs with column means
col_means = np.nanmean(X_train, axis=0)
inds = np.where(np.isnan(X_train))
X_train[inds] = np.take(col_means, inds[1])

inds_test = np.where(np.isnan(X_test))
X_test[inds_test] = np.take(col_means, inds_test[1])

# Evaluate MAML model directly on X_test
with torch.no_grad():
    maml_predictions = modell(torch.tensor(X_test, dtype=torch.float32))
    maml_predictions = torch.argmax(maml_predictions, dim=1).numpy()  # Convert to numpy array

# Calculate accuracy for MAML
maml_accuracy = accuracy_score(y_test, maml_predictions)

# Train other classifiers for comparison
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)
svm_predictions = svm_classifier.predict(X_test)

log_reg_classifier = LogisticRegression(max_iter=1000)
log_reg_classifier.fit(X_train, y_train)
log_reg_predictions = log_reg_classifier.predict(X_test)

# Step 7: Evaluate all classifiers
svm_accuracy = accuracy_score(y_test, svm_predictions)
log_reg_accuracy = accuracy_score(y_test, log_reg_predictions)

# Print the results
print(f"MAML Model Accuracy: {maml_accuracy}")
print(f"SVM Accuracy: {svm_accuracy}")
print(f"Logistic Regression Accuracy: {log_reg_accuracy}")


CONSIDERING MAML MODEL AS IT PERFORMED WELL

In [ ]:
# Initialize counters for model's prediction of class 1
c1 = 0
# Iterate through each prediction sample
for idx, x in enumerate(X_predict):
    x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(0)  # Add batch dimension for MAML model

    # Perform prediction with MAML model
    with torch.no_grad():
        maml_output = modell(x_tensor)
        maml_pred = torch.argmax(maml_output, dim=1).item()  # Get the predicted class as an integer
    if maml_pred == 1:
        c1 += 1

    # Print the predictions for this sample
    print(f"Sample {papers_data[i]['paper_id']}: MAML={maml_pred}")

# Print the total counts for class 1 predictions
print(f"\nSummary of predictions for class 1:")
print(f"MAML: {c1}")


-----------------------------------------------**TASK 2**--------------------------------------------

ARRANGING ALL THE REFRENCE PAPER ACCORDING TO THE CONFERENCE

In [ ]:
conferences = [
      {
          'name': 'TMLR',
          'reference_papers': [
              {'file_name': 'TMLR_paper1.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/KDSH_2025_Dataset/Reference/Publishable/TMLR/R014.pdf'},
              {'file_name': 'TMLR_paper2.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/KDSH_2025_Dataset/Reference/Publishable/TMLR/R015.pdf'}
          ]
      },
      {
          'name': 'NeurIPS',
          'reference_papers': [
              {'file_name': 'NeurIPS_paper1.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/KDSH_2025_Dataset/Reference/Publishable/NeurIPS/R012.pdf'},
              {'file_name': 'NeurIPS_paper2.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/KDSH_2025_Dataset/Reference/Publishable/NeurIPS/R013.pdf'}
          ]
      },
      {
          'name': 'KDD',
          'reference_papers': [
              {'file_name': 'KDD_paper1.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/KDSH_2025_Dataset/Reference/Publishable/KDD/R010.pdf'},
              {'file_name': 'KDD_paper2.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/KDSH_2025_Dataset/Reference/Publishable/KDD/R011.pdf'}
          ]
      },
      {
          'name': 'EMNLP',
          'reference_papers': [
              {'file_name': 'EMNLP_paper1.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/KDSH_2025_Dataset/Reference/Publishable/EMNLP/R008.pdf'},
              {'file_name': 'EMNLP_paper2.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/KDSH_2025_Dataset/Reference/Publishable/EMNLP/R009.pdf'}
          ]
      },
      {
          'name': 'CVPR',
          'reference_papers': [
              {'file_name': 'CVPR_paper1.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/KDSH_2025_Dataset/Reference/Publishable/CVPR/R006.pdf'},
              {'file_name': 'CVPR_paper2.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/KDSH_2025_Dataset/Reference/Publishable/CVPR/R007.pdf'}
          ]
      }
  ]


INTIALIZING WITH THE HUGGINGFACE API TOKEN

not sharing the token along with code to keep it private

In [ ]:
HUGGINGFACE_API_TOKEN = "....API TOKEN....."

SEVERAL OPERATIONS TO LEARN PREDICT THE CONFERENCE ALONG WITH RATIONALE

In [ ]:
#Initializing important urls
SUMMARIZATION_MODEL_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
EMBEDDING_MODEL_URL = "https://api-inference.huggingface.co/models/sentence-transformers/all-MiniLM-L6-v2"
TEXT_GENERATION_MODEL_URL = "https://api-inference.huggingface.co/models/EleutherAI/gpt-neo-2.7B"
headers = {"Authorization": f"Bearer {HUGGINGFACE_API_TOKEN}"}
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
MAX_TOKENS = 512
model = SentenceTransformer('all-MiniLM-L6-v2')

def extract_structured_text(pdf_path):
    """
    Extract structured text from a PDF.
    """
    doc = fitz.open(pdf_path)
    structured_content = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        # Extract text as dictionary with more structured details
        text = page.get_text("dict")

        structured_content.append(text)
    return structured_content

def extract_and_flatten_text(structured_content):
    """
    Extract and flatten text content from structured content dictionaries.
    """
    extracted_text = []
    try:
        for page in structured_content:
            if 'blocks' in page:
                print(f"Processing page with {len(page['blocks'])} blocks.")  # Debug: Check number of blocks
                for block in page['blocks']:
                    if 'lines' in block:
                        print(f"Processing block with {len(block['lines'])} lines.")  # Debug: Check number of lines in each block
                        for line in block['lines']:
                            for span in line.get('spans', []):
                                text = span.get('text', '').strip()
                                if text:
                                    extracted_text.append(text)
            else:
                print("No blocks found on this page.")  # Debug: Check if blocks exist
    except Exception as e:
        print(f"Error during text extraction: {e}")
    print(extracted_text[0])
    return " ".join(extracted_text)  # Combine all text into a single string

def clean_text(content):
    """
    Clean the extracted text by removing extra spaces and non-ASCII characters.
    """
    try:
        if isinstance(content, str):
            return ' '.join(content.split()).encode('ascii', 'ignore').decode('utf-8')
        else:
            raise ValueError("Input content is not a valid string.")
    except Exception as e:
        print(f"Error during text cleaning: {e}")
        return ""
def split_text_into_chunks(text, max_token_length=1024):
    tokens = tokenizer(text, return_tensors="pt", truncation=False, padding=False)
    tokenized_text = tokens['input_ids'][0]
    print(f"len of tokenized text{len(tokenized_text)}")
    chunks = []

    # Split text into chunks of max_token_length tokens
    for i in range(0, len(tokenized_text), max_token_length):
        print(i)
        chunk = tokenized_text[i:i + max_token_length]
        chunks.append(tokenizer.decode(chunk, skip_special_tokens=True))

    return chunks
def summarize_content(content):
    """
    Summarize the given content using an external summarization API.
    """
    content = clean_text(content)
    print(f"the content..........{content[:50]}")
    print(len(content))  # Ensure the content is cleaned
    if not content:
        print("Empty content after cleaning. Skipping summarization.")
        return None

    extracted_text = content

    # Split the extracted text into chunks
    chunks = split_text_into_chunks(extracted_text)
    print(f"len of chunks{len(chunks)}")
    # Summarize each chunk
    summarized_text = ""
    i=0
    for chunk in chunks:
        print(f"Processing chunk {i}")
        print(chunk)
        print(type(chunk))
        i+=1
        try:
            payload = {"inputs": chunk}
            response = requests.post(SUMMARIZATION_MODEL_URL, headers=headers, json=payload)
            response.raise_for_status()  # Raise an error for non-200 status codes
            result = response.json()
            if isinstance(result, dict) and "error" in result:
                raise ValueError(f"Summarization API Error: {result['error']}")
            summarized_text +=result[0]["summary_text"]+" "
        except requests.exceptions.RequestException as e:
            print(f"Request error: {e}")
        except ValueError as ve:
            print(ve)
        except Exception as e:
            print(f"Unexpected error during summarization: {e}")
    return summarized_text

def process_pdf(pdf_path):
    """
    Process a PDF file: extract, clean, and summarize text.
    """
    try:
        structured_content = extract_structured_text(pdf_path)  # Step 1: Extract structured content
        print(f"Structured content extracted: {structured_content[:1]}")  # Debug: Print a sample

        flat_text = extract_and_flatten_text(structured_content)  # Step 2: Flatten text
        if not flat_text:
            print("No text extracted from the PDF.")
            return None
        print(f"Type of flat_text: {type(flat_text)}")
        print(f"First 100 characters of flat_text: {flat_text[:100]}")
        summary = summarize_content(flat_text)  # Step 3: Summarize content
        if summary:
            print(f"Summary generated: {summary[:100]}...")  # Debug: Print the first 100 characters
        else:
            print("Summary could not be generated.")
        return summary
    except Exception as e:
        print(f"Error during PDF processing: {e}")
        return None

def get_embedding_from_local_model(text):
    """
    Get the embedding for the text using the local SentenceTransformer model.
    """
    model = SentenceTransformer('all-MiniLM-L6-v2')
    # Get the embedding for the text
    print(type(model))
    embedding = model.encode(text)
    return np.array(embedding)


def get_text_embeddings(text, max_chunk_size=384):
    """
    Split the text into chunks, get embeddings via API, and combine them.
    """
    # Tokenize the text into chunks
    words = text.split()
    chunks = [' '.join(words[i:i + max_chunk_size]) for i in range(0, len(words), max_chunk_size)]

    chunk_embeddings = []
    print(len(chunks))
    i=0
    for chunk in chunks:
      print(f"Processing chunk {i}")
      i+=1
      # Get embedding for each chunk via API
      embedding = get_embedding_from_local_model(chunk)
      chunk_embeddings.append(embedding)

    # Combine all chunk embeddings using mean pooling
    combined_embedding = np.mean(chunk_embeddings, axis=0)
    return combined_embedding

# Debugging: Ensure proper inputs to clean_text and summarize_content
def debug_inputs(conferences):
    for i, conference in enumerate(conferences):
        if not isinstance(conference, str):
            print(f"Debug: Non-string content at index {i}: {conference}")
# Train profiles for conferences
def create_conference_profiles(conferences):
    profiles = {}
    i=0
    for conference in conferences:
        print(f"...................{i}....................")#for verifying the process
        i+=1
        print(f"Processing conference: {conference['name']}")
        embeddings = []
        for paper in conference['reference_papers']:
            # structured_text = extract_structured_text(paper['file_path'])
            summarized_text = process_pdf(paper['file_path'])
            embeddings.append(get_text_embeddings(summarized_text))
        profiles[conference['name']] = np.mean(embeddings, axis=0)
    return profiles

# Classify a test paper
def classify_test_paper(test_paper_path, conference_profiles):
    #structured_text = extract_structured_text(test_paper_path)
    summarized_text = process_pdf(test_paper_path)
    test_embedding = get_text_embeddings(summarized_text)

    scores = {}
    for conference, profile_embedding in conference_profiles.items():
        scores[conference] = cosine_similarity([test_embedding], [profile_embedding])[0][0]
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

def generate_justification(test_paper, best_conference, similarity_score):
    """
    Generates a detailed and concise justification for the classification of a paper to a conference.

    Parameters:
        test_paper (str): The content of the test paper.
        best_conference (str): The classified conference name.
        similarity_score (float): The similarity score for the classification.

    Returns:
        str: The generated justification.
    """
    prompt = (
        f"The paper is classified under the {best_conference} conference based on a similarity score of {similarity_score:.2f}. "
        f"Analyze the paper’s content, methodology, and findings, and provide a concise rationale (up to 100 words) explaining "
        f"how it aligns with the themes, focus areas, and quality standards of the {best_conference} conference."
    )
    payload = {"inputs": prompt}

    # Make a POST request to the Hugging Face text generation API
    response = requests.post(TEXT_GENERATION_MODEL_URL, headers=headers, json=payload)

    if response.status_code == 200:
        try:
            # Extract and return the generated text
            generated_text = response.json()[0]["generated_text"]
            generated_text = generated_text[len(prompt):].strip()
            # Limit to 100 words if the model generates more
            word_limit = 100
            truncated_text = " ".join(generated_text.split()[:word_limit])
            return truncated_text
        except (KeyError, IndexError) as e:
            raise ValueError(f"Unexpected response structure: {response.json()}") from e

    # Raise an error if the response is not successful
    raise ValueError(f"Error in text generation API: {response.text}")


INTIALIZING CONFERENCE PROFILE ALONG WITH PAPER IDS

In [ ]:
#storing  paper id
import time

profiles = create_conference_profiles(conferences)
print("-------CONFRENCE PROFILE COMPLETED----------")
paper_ids=[]
for paper in papers_data:
   paper_ids.append(paper["paper_id"])
# List to store results
print(paper_ids[:5])

PREDICTING THE CONFRENCE ALONG WITH PREDICTING IF THE PAPER IS PUBLISHABLE OR NOT

In [ ]:
results = []
# Predict one by one and store the results
for x in (X_predict):
  print(f"...................{i}....................")
  print(f"Processing paper {paper_ids[i]}")
  x_tensor = torch.tensor(x, dtype=torch.float32)  # Add batch dimension for MAML model
  with torch.no_grad():
      maml_output = modell(x_tensor)
      maml_pred = torch.argmax(maml_output, dim=0).numpy()
  if(maml_pred==1):
    test_paper_text=os.path.join(papers_path,papers_data[i]['paper_id']+".pdf" )
    classification = classify_test_paper(test_paper_text, profiles)
    best_conference, score = classification[0]
    # Generate justification
    justification = generate_justification(test_paper_text, best_conference, score)
    print(f"Paper ID: {paper_ids[i]}, MAML Prediction: {maml_pred}, Classified Conference: {best_conference}, Rationale: {justification}")
    results.append([paper_ids[i], maml_pred, best_conference,justification])
  # Add result to the list
  else:
    results.append([paper_ids[i], maml_pred,"na","na"])




SAVING THE RESULT

In [ ]:
print(len(results))# to verify every test papers are iterated
df = pd.DataFrame(results, columns=['Paper ID', 'Publishable','Conference','Rationale'])

# Save DataFrame to CSV
df.to_csv('predictions.csv', index=False)

print("Predictions saved to 'predictions.csv'")
